In [36]:
import cv2
import numpy as np
import pandas as pd
import time
import random
import matplotlib.pyplot as plt

In [3]:
# Download data from https://www.kaggle.com/datasets/fmena14/crowd-counting/code
# data size too big cannot updload to github
images = np.load("../data/images.npy")      # size: 640*480
label = np.load("../data/labels.npy")

In [ ]:
# Wider Pedestrian Dataset (https://drive.google.com/file/d/1I7OjhaomWqd8Quf7o5suwLloRlY0THbp/view)
# extension of the Wider Face dataset and focuses on pedestrian detection


# Default HogDescriptor

In [8]:
def count_people(image):
    # Load the pre-trained HOG people detector
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    # Convert the image to grayscale for HOG detector
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect people in the image
    people, _ = hog.detectMultiScale(gray)

    # Return the count of detected people
    return len(people)

In [9]:
countErr = []
for img, lab in zip(images[:100], label[:100]):
    count = count_people(img)
    error = lab[0] - count
    countErr.append([count, error])

In [11]:
result = np.append(label[:100], countErr, axis=1)
df = pd.DataFrame(result, columns=["actual", "counted", "error"])
print("Average Error: ", sum(df["error"])/len(df["error"]))

,actual,counted,error
0,35,8,27
1,41,9,32
2,41,6,35
3,44,7,37
4,41,14,27
...,...,...,...
95,25,5,20
96,28,7,21
97,30,6,24
98,31,5,26


# Scale image
The HOG detector may perform better at different scales. Try detecting people at multipel scales by resizing the image

In [52]:
def count_people(image, scale):
    # Load the pre-trained HOG people detector
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    scaled_image = cv2.resize(image, (int(image.shape[1] * scale), int(image.shape[0] * scale)))

    # Convert the image to grayscale for HOG detector
    gray = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2GRAY)

    # Detect people in the image
    people, _ = hog.detectMultiScale(gray)

    # Return the count of detected people
    return len(people)

In [53]:
scaleList = np.arange(1.0, 3.0, 0.2)
countErr = {scale:[] for scale in scaleList}
for _ in range(100):
    i = random.randint(0, len(images))
    img = images[i]
    lab = label[i]
    for scale in scaleList:
        startTime = time.time()
        count = count_people(img, scale)
        elapsedTime = time.time() - startTime
        error = ((lab[0] - count) / lab[0]) * 100
        countErr[scale].append([count, error, elapsedTime])

In [54]:
scaleAnalysis = pd.DataFrame(columns=["scale", "average percentage error", "average computing time"])
i = 0
for scale, count in countErr.items():
    df = pd.DataFrame(count, columns=["count", "error", "time"])
    err = round(sum(df["error"])/len(df["error"]), 3)
    computeTime = round(sum(df["time"])/len(df["time"]), 3)
    scaleAnalysis.loc[i] = [round(scale, 2), err, computeTime]
    i += 1
scaleAnalysis

,scale,average percentage error,average computing time
0,1.0,81.977,0.099
1,1.2,73.362,0.156
2,1.4,66.874,0.239
3,1.6,63.203,0.322
4,1.8,61.072,0.430
5,2.0,58.754,0.536
6,2.2,56.657,0.659
7,2.4,54.867,0.817
8,2.6,55.178,0.942
9,2.8,53.972,1.105


# winStride and padding
Scaling is working. As increasing the size of image, the model is taking longer time but definitely reducing the error. To keep fast computing time, will use scaling between 1.8 and 2.2 and adjust other parameters such as winStride and padding  

help: https://stackoverflow.com/questions/26607418/improving-accuracy-opencv-hog-people-detector

In [58]:
def count_people(image, winStride, padding):
    # Load the pre-trained HOG people detector
    hog = cv2.HOGDescriptor(_winSize=(64, 128), _blockSize=(16, 16), _blockStride=(8, 8), _cellSize=(8, 8), _nbins=9)
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    scaled_image = cv2.resize(image, (int(image.shape[1] * 2.2), int(image.shape[0] * 2.2)))

    # Convert the image to grayscale for HOG detector
    gray = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2GRAY)

    # Detect people in the image
    people, _ = hog.detectMultiScale(gray, winStride=winStride, padding=padding)

    # Return the count of detected people
    return len(people)

In [59]:
winStrideList = [(4,4), (4,8), (8,8), (8,16), (16, 16), (16, 32), (32, 32)]
countErr = {ws:[] for ws in winStrideList}

for _ in range(100):
    i = random.randint(0, len(images))
    img = images[i]
    lab = label[i]
    for winStride in winStrideList:
        startTime = time.time()
        count = count_people(img, winStride, (8,8))
        elapsedTime = time.time() - startTime
        error = ((lab[0] - count) / lab[0]) * 100
        countErr[winStride].append([count, error, elapsedTime])

In [60]:
winAnalysis = pd.DataFrame(columns=["winStride", "average percentage error", "average computing time"])
i = 0
for ws, count in countErr.items():
    df = pd.DataFrame(count, columns=["count", "error", "time"])
    err = round(sum(df["error"])/len(df["error"]), 3)
    computeTime = round(sum(df["time"])/len(df["time"]), 3)
    winAnalysis.loc[i] = [ws, err, computeTime]
    i += 1
winAnalysis

,winStride,average percentage error,average computing time
0,"(4, 4)",38.693,2.254
1,"(4, 8)",47.725,1.108
2,"(8, 8)",57.705,0.571
3,"(8, 16)",67.026,0.388
4,"(16, 16)",80.638,0.297
5,"(16, 32)",94.271,0.249
6,"(32, 32)",99.085,0.226
